In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# load a list of abnormal addressess
abnormal_addresses_list = json.load(open("kmeans-outliers.json", "r"))

In [3]:
df_chunk = pd.read_csv('transaction.csv', chunksize=1000000, iterator = True)
chunk_list = [] 

for chunk in df_chunk:
    chunk_list.append(chunk)

df = pd.concat(chunk_list) #再把这些块组合成一个DataFrame
df["amount"] = df["amount"].astype("float") / 10**18
df

,tx_hash,block_num,from_addr,to_addr,amount
0,0x4372648d4742446b1efe06f8e27eee2928cd37418a6a...,5358831,0xee1087889193c4acc92c5bfec090c0a43a7b132f,0xea355aefd533a5df887d8f6d16da75373d260719,10.998620
1,0x4631a157efb62403573051be989b8e4e02a846e03db8...,5358831,0x30146933a3a0babc74ec0b3403bec69281ba5914,0x52146615ac40deebd6913d05e8b6fbfec2858dab,0.430000
2,0xf6ba7bb5a91d70765542eadeaf6688f2ee1dbc65876a...,5358831,0x30146933a3a0babc74ec0b3403bec69281ba5914,0xb38142943607d6fcef5932777d2286831fbe8fca,1.507000
3,0xd9013be05615e68347fcb1751dfef6ae8c271d6a6dd8...,5358831,0x30146933a3a0babc74ec0b3403bec69281ba5914,0x852b13174d30ffda834db573bfb00fec955b43e8,0.129800
4,0x76f9a7e8b09e04854819b4208ed0a18ad99fb1ff5c06...,5358831,0x79fb89a976c831a903992a9166ea1a3102b0caef,0x7f5f2922a21b0e9a63d9ec94882ef612a8e01f0a,0.045287
...,...,...,...,...,...
10779102,0xd4585b8eeff60772d7ab429f2e6e4586e78d2d7430ff...,5535804,0xddd3fc4a0f9e3f2a966609f8a329900cb30a4eaa,0xea982934a654c7fb391babc5e92286e7acfb059e,0.036487
10779103,0x440673c013684d99412bca3f4a923126f637f7896d63...,5535804,0xad68942a95fdd56594aa5cf862b358790e37834c,0x456150c08ad4ae665c1012aae582b778ce8c145c,0.000111
10779104,0x6a57e561593c21e9832a3b16667c6da47b3491d55992...,5535804,0x29496e4c1ee9d6ea923af1ddfe872c1393605827,0xf683306b7607cc8eed34555e5fd69717dc96373b,0.000300
10779105,0xee0ae70d6451014283573da62b8fad4f6a3ff8cfcf70...,5535804,0x96dd3a75fe4495ad71065205c1a37785390a26b6,0xc28e860c9132d55a184f9af53fc85e90aa3a0153,0.122000


In [4]:
# get a list of addresses
addresses = set(df["from_addr"].unique())
addresses.update(df["to_addr"].unique())
addresses_df = pd.Series(np.asarray(list(addresses), dtype=np.chararray)).to_frame(name="address")
addresses_df["id"] = addresses_df.index
addresses_df.set_index(addresses_df.address, inplace=True)
addresses_df = addresses_df.drop(["address"], axis=1)
addresses_df

,id
address,
0x9977e4f6b9434e2775d43fbb86f6d8d46f11a6f1,0
0x3dedb7bc61452309c52c852471eceb88537ee498,1
0x6f0a214195fc78c6c88a3a2e7dfabfa01d0540ad,2
0x7d0970f6a5a807b662ee27f848aaca5dcdb857be,3
0xf6ac876e325ab0b5d9ef5fd77d743951c30e1131,4
...,...
0x4f8771c42f53e5f617c2e18ee883f0f5965b41ea,3653827
0xd125e31c3d289cabc401a15cbec2e66898b0fe0e,3653828
0xcac18d760a8aa073341395ca9462daf4cc0d6989,3653829


In [6]:
addresses_df["type"] = 0
addresses_df.type.loc[abnormal_addresses_list] = 1
nodes_df = addresses_df.reset_index()
nodes_df.rename({"id": "id", "address": "addr"}, inplace=True)
nodes_df[":LABEL"] = "Address"
nodes_df

,address,id,type,:LABEL
0,0x9977e4f6b9434e2775d43fbb86f6d8d46f11a6f1,0,0,Address
1,0x3dedb7bc61452309c52c852471eceb88537ee498,1,0,Address
2,0x6f0a214195fc78c6c88a3a2e7dfabfa01d0540ad,2,0,Address
3,0x7d0970f6a5a807b662ee27f848aaca5dcdb857be,3,0,Address
4,0xf6ac876e325ab0b5d9ef5fd77d743951c30e1131,4,0,Address
...,...,...,...,...
3653827,0x4f8771c42f53e5f617c2e18ee883f0f5965b41ea,3653827,0,Address
3653828,0xd125e31c3d289cabc401a15cbec2e66898b0fe0e,3653828,0,Address
3653829,0xcac18d760a8aa073341395ca9462daf4cc0d6989,3653829,0,Address
3653830,0xfdb9dfef782e7d999a7990a04652acd46d7b6d71,3653830,0,Address


In [7]:
from_to_agg_df = df[["from_addr", "to_addr", "amount"]].groupby(
    ["from_addr", "to_addr"]
    ).agg(
        total_amount=pd.NamedAgg(column='amount', aggfunc='sum'), 
        count=pd.NamedAgg(column='amount', aggfunc='count')
    )
from_to_agg_df.reset_index(inplace=True)
from_to_agg_df

,from_addr,to_addr,total_amount,count
0,0x000000008061205ec0a4ce8cb519ba56bdf44fbc,0x6090a6e47849629b7245dfa1ca21d94cd15878ef,0.010000,1
1,0x000000008061205ec0a4ce8cb519ba56bdf44fbc,0x7b6a49333e89bef730dbad026708f6d160fe3834,0.008000,2
2,0x000000043dc3052d771845a71efc05b67f40abb4,0x4491c37ff1b5ec95d4401d9413679034743ace53,0.035000,1
3,0x0000000484f2217f1a64eb6d24b5cee446faeae5,0x009e02b21abefc7ecc1f2b11700b49106d7d552b,1.770000,1
4,0x0000000484f2217f1a64eb6d24b5cee446faeae5,0xdf31215ec3b8e3c0e0e1e17fc28711ec9c839c2c,17.060000,1
...,...,...,...,...
5438887,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0x4ab57165aa07990cd4d120b6978b0c70d55be7cd,10.000000,7
5438888,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0x4fe7175154f8111ea656e1cd77531b8043b916ec,2.000000,2
5438889,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0xca195a5e1842711e8b1ecda923534cded106b8e9,2.000000,1
5438890,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0xe23152d029d233d9d8efe81b18f0ef6dd3364317,1.000000,1


In [8]:
# join the two dataframes
from_to_agg_df[":START_ID"] = pd.merge(from_to_agg_df["from_addr"], addresses_df,
                                    left_on="from_addr", right_index=True).id
from_to_agg_df[":END_ID"] = pd.merge(from_to_agg_df["to_addr"], addresses_df,
                                    left_on="to_addr", right_index=True).id
from_to_agg_df

,from_addr,to_addr,total_amount,count,:START_ID,:END_ID
0,0x000000008061205ec0a4ce8cb519ba56bdf44fbc,0x6090a6e47849629b7245dfa1ca21d94cd15878ef,0.010000,1,2868012,2749588
1,0x000000008061205ec0a4ce8cb519ba56bdf44fbc,0x7b6a49333e89bef730dbad026708f6d160fe3834,0.008000,2,2868012,863488
2,0x000000043dc3052d771845a71efc05b67f40abb4,0x4491c37ff1b5ec95d4401d9413679034743ace53,0.035000,1,529924,603240
3,0x0000000484f2217f1a64eb6d24b5cee446faeae5,0x009e02b21abefc7ecc1f2b11700b49106d7d552b,1.770000,1,2663813,970555
4,0x0000000484f2217f1a64eb6d24b5cee446faeae5,0xdf31215ec3b8e3c0e0e1e17fc28711ec9c839c2c,17.060000,1,2663813,1170178
...,...,...,...,...,...,...
5438887,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0x4ab57165aa07990cd4d120b6978b0c70d55be7cd,10.000000,7,355305,684711
5438888,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0x4fe7175154f8111ea656e1cd77531b8043b916ec,2.000000,2,355305,3545921
5438889,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0xca195a5e1842711e8b1ecda923534cded106b8e9,2.000000,1,355305,618127
5438890,0xfffffd2b4cbf69d99a7997bc3543191c79acdb18,0xe23152d029d233d9d8efe81b18f0ef6dd3364317,1.000000,1,355305,2936672


In [10]:
edges_df = from_to_agg_df[[":START_ID", "count", "total_amount", ":END_ID"]]
edges_df[":LABEL"] = "SENT"

/var/folders/4c/y7d9384d7g17636gh79f9zjc0000gn/T/ipykernel_73067/1888948222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges_df[":LABEL"] = "SENT"


In [11]:
nodes_df.to_csv("nodes.csv", index=False)
edges_df.to_csv("edges.csv", index=False)